In [1]:
import pandas as pd
import os
from Bio import Entrez, SeqIO
import tqdm
from collections import Counter
import numpy as np
from scipy.stats import multinomial
import numpy as np


In [ ]:
# Load the data iteratively to avoid crashing the kernel
iterator = pd.read_csv("/Users/reem/Mov/nextclade_results/deduplicated_processed.tsv",sep="\t", chunksize=1000)
df = pd.concat([chunk for chunk in tqdm.tqdm(iterator, desc='Loading data')])
df.head()




Loading data: 17435it [00:51, 341.75it/s]


,Unnamed: 0,seqName,privateNucMutations.unlabeledSubstitutions,subs,Counts,context,spectrum,G>A_context_counts,proportions
0,0,hCoV-19/USA/MO-WRAIR-COX5040NPS/2020|2020-08-1...,"G204A,C2445T,C4331T,C5621T,C5622T,C6633T,C1240...","G>A,C>T,C>T,C>T,C>T,C>T,C>T,G>A,C>T,A>C,A>T,G>...","{'G>A': 3, 'C>T': 8, 'A>C': 1, 'A>T': 1, 'G>T'...","CGT,ACT,TCT,ACC,CCT,GCT,TCA,AGG,TCA,GAG,CAA,TG...","C[G>A]T,A[C>T]T,T[C>T]T,A[C>T]C,C[C>T]T,G[C>T]...","Counter({'C[G>A]T': 1, 'A[G>A]G': 1, 'T[G>A]C'...","{'C[G>A]T': 0.3333333333333333, 'A[G>A]G': 0.3..."
1,1,hCoV-19/Belgium/UGent-14493/2021|2021-12-22|20...,NaN,NaN,{},NaN,NaN,Counter(),{}
2,2,hCoV-19/France/IDF-HMN-21052200412/2021|2021-0...,NaN,NaN,{},NaN,NaN,Counter(),{}
3,3,hCoV-19/England/LSPA-37EF052/2022|2022-02-21|2...,G19999T,G>T,{'G>T': 1},AGT,A[G>T]T,Counter(),{}
4,4,hCoV-19/Germany/BE-RKI-I-595719/2022|2022-02-2...,"G410T,A7881G","G>T,A>G","{'G>T': 1, 'A>G': 1}","TGG,AAT","T[G>T]G,A[A>G]T",Counter(),{}


In [9]:
df_Mov = df[df["LLR"]>6]
df_Mov.head()

KeyError: 'LLR'

In [30]:
df_Normal = df[df["LLR"]<6]
df_Normal.head()

,Unnamed: 0,seqName,C[G>A]T,A[G>A]G,T[G>A]C,A[G>A]C,C[G>A]G,T[G>A]G,G[G>A]T,A[G>A]A,T[G>A]T,A[G>A]T,G[G>A]C,G[G>A]G,T[G>A]A,C[G>A]C,G[G>A]A,C[G>A]A,LLR
0,0,hCoV-19/USA/MO-WRAIR-COX5040NPS/2020|2020-08-1...,0.333333,0.333333,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.378720
1,1,hCoV-19/Belgium/UGent-14493/2021|2021-12-22|20...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,2,hCoV-19/France/IDF-HMN-21052200412/2021|2021-0...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,3,hCoV-19/England/LSPA-37EF052/2022|2022-02-21|2...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.509730
4,4,hCoV-19/Germany/BE-RKI-I-595719/2022|2022-02-2...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.720178


In [31]:
mean_row = df_Mov.drop(columns = ["seqName","LLR"]).mean()
mean_row["seqName"] = "Mean"
mean_row["LLR"] = "None"
mean_row = mean_row[df_Mov.columns]
df_Mov = pd.concat([df_Mov,pd.DataFrame([mean_row])],ignore_index=True)
print(df_Mov.tail(10))


       Unnamed: 0                                            seqName  \
745  1.726580e+07  hCoV-19/Slovakia/TN_00736/2022|2022-03-08|2022...   
746  1.727268e+07  hCoV-19/USA/CA-CDPH-1002014907/2023|2023-09-12...   
747  1.728780e+07  hCoV-19/Thailand/CU-AMV11/2024|2024-02-05|2024...   
748  1.731203e+07  hCoV-19/Denmark/DCGC-466201/2022|2022-03-28|20...   
749  1.731997e+07  hCoV-19/Vietnam/BMH-310322-sample88/2022|2022-...   
750  1.733009e+07  hCoV-19/England/PHEP-YYFYNOA/2022|2022-05-25|2...   
751  1.733937e+07  hCoV-19/Australia/NSW-SAVID-9765/2022|2022-08-...   
752  1.738204e+07  hCoV-19/USA/TX-HMH-M-106148/2022|2022-07-08|20...   
753  1.739842e+07  hCoV-19/England/LSPA-3B50559/2022|2022-03-17|2...   
754  8.527270e+06                                               Mean   

      C[G>A]T   A[G>A]G   T[G>A]C   A[G>A]C   C[G>A]G   T[G>A]G   G[G>A]T  \
745  0.000000  0.000000  0.222222  0.000000  0.000000  0.222222  0.111111   
746  0.000000  0.000000  0.222222  0.000000  0.000000

In [32]:
context_cols = [col for col in df_Mov.columns if col not in ["seqName", "LLR","Unnamed: 0"]]
mean_row = df_Mov.iloc[-1]
mean_row = mean_row[context_cols]
print(mean_row)

df_probs = mean_row.reset_index()
df_probs.columns = ['Context', 'Molnupiravir']
df_probs

C[G>A]T    0.056561
A[G>A]G    0.025235
T[G>A]C    0.161416
A[G>A]C    0.046691
C[G>A]G    0.015436
T[G>A]G    0.064542
G[G>A]T    0.037379
A[G>A]A    0.032245
T[G>A]T    0.261609
A[G>A]T    0.063791
G[G>A]C    0.029353
G[G>A]G    0.010296
T[G>A]A    0.095772
C[G>A]C    0.019442
G[G>A]A    0.020389
C[G>A]A    0.018729
Name: 754, dtype: object


,Context,Molnupiravir
0,C[G>A]T,0.056561
1,A[G>A]G,0.025235
2,T[G>A]C,0.161416
3,A[G>A]C,0.046691
4,C[G>A]G,0.015436
5,T[G>A]G,0.064542
6,G[G>A]T,0.037379
7,A[G>A]A,0.032245
8,T[G>A]T,0.261609
9,A[G>A]T,0.063791


In [33]:
mean_row = df_Normal.drop(columns = ["seqName","LLR"]).mean()
mean_row["seqName"] = "Mean"
mean_row["LLR"] = "None"
mean_row = mean_row[df_Normal.columns]
df_Normal = pd.concat([df_Normal,pd.DataFrame([mean_row])],ignore_index=True)
print(df_Normal.tail(10))


            Unnamed: 0                                            seqName  \
17433862  1.743462e+07  hCoV-19/USA/CA-CDC-ASC210155423/2021|2021-07-0...   
17433863  1.743462e+07  hCoV-19/France/ARA-GENBIO-14292281237/2022|202...   
17433864  1.743462e+07  hCoV-19/Israel/ICH-741184712/2022|2022-08-13|2...   
17433865  1.743462e+07  hCoV-19/USA/MD-CDC-QDX35868572/2022|2022-04-11...   
17433866  1.743462e+07  hCoV-19/Sweden/01_SE100_22CS105674/2022|2022-0...   
17433867  1.743462e+07  hCoV-19/USA/AZ-ASU89690/2022|2022-08-09|2022-0...   
17433868  1.743462e+07  hCoV-19/England/CAMC-11E8DE5/2021|2021-02-03|2...   
17433869  1.743462e+07  hCoV-19/Australia/VIC49129/2022|2022-03-28|202...   
17433870  1.743462e+07  hCoV-19/USA/IN-CDC-ASC210483853/2021|2021-11-2...   
17433871  8.717320e+06                                               Mean   

           C[G>A]T   A[G>A]G  T[G>A]C   A[G>A]C   C[G>A]G   T[G>A]G   G[G>A]T  \
17433862  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000  0.

In [6]:
context_cols = [col for col in df_Normal.columns if col not in ["seqName", "LLR","Unnamed: 0"]]
mean_row = df_Normal.iloc[-1]
#print(mean_row)
mean_row = mean_row[context_cols]

df2 = mean_row.reset_index()
df2.columns = ['Context', 'Normal']

df_probs['Normal'] = df2['Normal'].values

df_probs

NameError: name 'df_Normal' is not defined

In [17]:
iterator = pd.read_csv("/Users/reem/Mov/nextclade_results/counts_final_results.tsv", sep="\t", chunksize=1000)
df = pd.concat([chunk for chunk in tqdm.tqdm(iterator, desc='Loading data')])
df.head()

Loading data: 1415it [00:03, 450.84it/s]


,Unnamed: 0,seqName,privateNucMutations.unlabeledSubstitutions,subs,Counts,context,spectrum,context_counts
0,0,hCoV-19/USA/MO-WRAIR-COX5040NPS/2020|2020-08-1...,"G204A,C2445T,C4331T,C5621T,C5622T,C6633T,C1240...","G>A,C>T,C>T,C>T,C>T,C>T,C>T,G>A,C>T,A>C,A>T,G>...","{'G>A': 3, 'C>T': 8, 'A>C': 1, 'A>T': 1, 'G>T'...","CGT,ACT,TCT,ACC,CCT,GCT,TCA,AGG,TCA,GAG,CAA,TG...","C[G>A]T,A[C>T]T,T[C>T]T,A[C>T]C,C[C>T]T,G[C>T]...","Counter({'C[G>A]T': 1, 'A[G>A]G': 1, 'T[G>A]C'..."
1,1,hCoV-19/Belgium/UGent-14493/2021|2021-12-22|20...,NaN,NaN,{},NaN,NaN,Counter()
2,2,hCoV-19/France/IDF-HMN-21052200412/2021|2021-0...,NaN,NaN,{},NaN,NaN,Counter()
3,3,hCoV-19/England/LSPA-37EF052/2022|2022-02-21|2...,G19999T,G>T,{'G>T': 1},AGT,A[G>T]T,Counter()
4,4,hCoV-19/Germany/BE-RKI-I-595719/2022|2022-02-2...,"G410T,A7881G","G>T,A>G","{'G>T': 1, 'A>G': 1}","TGG,AAT","T[G>T]G,A[A>G]T",Counter()


In [ ]:
df.context_counts.rename({'context_counts': 'G>A_context_counts'},inplace=True)

In [ ]:
#df.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
#df.head()
df.to_csv("/Users/reem/Mov/nextclade_results/counts_final_results.tsv", sep="\t")


In [14]:
df.head()

,Unnamed: 0,seqName,privateNucMutations.unlabeledSubstitutions,subs,Counts,context,spectrum,G>A_context_counts,proportions
0,0,hCoV-19/USA/MO-WRAIR-COX5040NPS/2020|2020-08-1...,"G204A,C2445T,C4331T,C5621T,C5622T,C6633T,C1240...","G>A,C>T,C>T,C>T,C>T,C>T,C>T,G>A,C>T,A>C,A>T,G>...","{'G>A': 3, 'C>T': 8, 'A>C': 1, 'A>T': 1, 'G>T'...","CGT,ACT,TCT,ACC,CCT,GCT,TCA,AGG,TCA,GAG,CAA,TG...","C[G>A]T,A[C>T]T,T[C>T]T,A[C>T]C,C[C>T]T,G[C>T]...","Counter({'C[G>A]T': 1, 'A[G>A]G': 1, 'T[G>A]C'...","{'C[G>A]T': 0.3333333333333333, 'A[G>A]G': 0.3..."
1,1,hCoV-19/Belgium/UGent-14493/2021|2021-12-22|20...,NaN,NaN,{},NaN,NaN,Counter(),{}
2,2,hCoV-19/France/IDF-HMN-21052200412/2021|2021-0...,NaN,NaN,{},NaN,NaN,Counter(),{}
3,3,hCoV-19/England/LSPA-37EF052/2022|2022-02-21|2...,G19999T,G>T,{'G>T': 1},AGT,A[G>T]T,Counter(),{}
4,4,hCoV-19/Germany/BE-RKI-I-595719/2022|2022-02-2...,"G410T,A7881G","G>T,A>G","{'G>T': 1, 'A>G': 1}","TGG,AAT","T[G>T]G,A[A>G]T",Counter(),{}


In [22]:
df_Mov =pd.read_csv("/Users/reem/Mov/nextclade_results/Mov_means.tsv", sep="\t")
df_Normal = pd.read_csv("/Users/reem/Mov/nextclade_results/Normal_means.tsv", sep="\t")
context_cols = [col for col in df_Normal.columns if col not in ["seqName","Unnamed: 0", "LLR"]]
mean_row_Normal = df_Normal.iloc[-1][context_cols]
mean_row_Mov = df_Mov.iloc[-1][context_cols]

df_probs = pd.DataFrame({
    'Context': context_cols,
    'Molnupiravir': mean_row_Mov.values,
    'Normal': mean_row_Normal.values})
df_probs['Context'] = context_cols
df_probs['Normal'] = df_probs['Normal'].values
df_probs['Molnupiravir'] = df_probs['Molnupiravir'].values
df_probs

,Context,Molnupiravir,Normal
0,C[G>A]T,0.054176,0.020609
1,A[G>A]G,0.025408,0.032408
2,T[G>A]C,0.160377,0.015557
3,A[G>A]C,0.049371,0.018432
4,C[G>A]G,0.015709,0.012249
5,T[G>A]G,0.064058,0.023983
6,G[G>A]T,0.038508,0.011146
7,A[G>A]A,0.031309,0.026299
8,T[G>A]T,0.26164,0.040269
9,A[G>A]T,0.066542,0.027112


In [27]:
import ast
df["G>A_context_counts"] = (
    df["G>A_context_counts"]
    .astype(str)
    .str.replace("Counter(", "")  # remove "Counter("
    .str.rstrip(")")                           # remove trailing ")"
    .apply(lambda x: ast.literal_eval(x) if x not in ["", "nan", "Counter"] else {})
)
print(df["G>A_context_counts"].head(20))
print(type(df["G>A_context_counts"]))

0     {'C[G>A]T': 1, 'A[G>A]G': 1, 'T[G>A]C': 1}
1                                             {}
2                                             {}
3                                             {}
4                                             {}
5                                             {}
6                                 {'A[G>A]C': 1}
7                                             {}
8                                             {}
9                                             {}
10                                            {}
11                                {'C[G>A]T': 1}
12                                {'A[G>A]G': 1}
13                                            {}
14                                            {}
15                                            {}
16                                            {}
17                                {'C[G>A]G': 1}
18                                {'T[G>A]G': 1}
19                                            {}
Name: G>A_context_co

In [28]:
def get_likelihood_ratio(counts,pM,pN):
    counts=np.array(counts,dtype=float)
    llM= float(multinomial.logpmf(counts, n=np.sum(counts), p=pM))
    llN = float(multinomial.logpmf(counts, n=np.sum(counts), p=pN))
    llr=llM-llN
    return llr


pM=df_probs["Molnupiravir"].values.tolist()
pN=df_probs["Normal"].values.tolist()
mutation_types = df_probs["Context"]


llr_list=[]
for i,row in df.iterrows():
    counts_dict = row["G>A_context_counts"]
    counts = [counts_dict.get(mut,0) for mut in mutation_types]
    llr = get_likelihood_ratio(counts,pM,pN)
    llr_list.append(llr)
df["LLR"] = llr_list
df["LLR"].head(20)


/var/folders/bt/jpy5j9ms7pb6p6hhqzvpjffw0000gn/T/ipykernel_2144/4122262617.py:3: FutureWarning: Some rows of `p` do not sum to 1.0 within tolerance of eps=np.float64(2.220446049250313e-15). Currently, the last element of these rows is adjusted to compensate, but this condition will produce NaNs beginning in SciPy 1.18.0. Please ensure that rows of `p` sum to 1.0 to avoid futher disruption.
  llM= float(multinomial.logpmf(counts, n=np.sum(counts), p=pM))
/var/folders/bt/jpy5j9ms7pb6p6hhqzvpjffw0000gn/T/ipykernel_2144/4122262617.py:4: FutureWarning: Some rows of `p` do not sum to 1.0 within tolerance of eps=np.float64(2.220446049250313e-15). Currently, the last element of these rows is adjusted to compensate, but this condition will produce NaNs beginning in SciPy 1.18.0. Please ensure that rows of `p` sum to 1.0 to avoid futher disruption.
  llN = float(multinomial.logpmf(counts, n=np.sum(counts), p=pN))


0     3.056241
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.985277
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.966537
12   -0.243338
13    0.000000
14    0.000000
15    0.000000
16    0.000000
17    0.248802
18    0.982433
19    0.000000
Name: LLR, dtype: float64

In [30]:
df.head(20)

,Unnamed: 0,seqName,privateNucMutations.unlabeledSubstitutions,subs,Counts,context,spectrum,G>A_context_counts,proportions,LLR
0,0,hCoV-19/USA/MO-WRAIR-COX5040NPS/2020|2020-08-1...,"G204A,C2445T,C4331T,C5621T,C5622T,C6633T,C1240...","G>A,C>T,C>T,C>T,C>T,C>T,C>T,G>A,C>T,A>C,A>T,G>...","{'G>A': 3, 'C>T': 8, 'A>C': 1, 'A>T': 1, 'G>T'...","CGT,ACT,TCT,ACC,CCT,GCT,TCA,AGG,TCA,GAG,CAA,TG...","C[G>A]T,A[C>T]T,T[C>T]T,A[C>T]C,C[C>T]T,G[C>T]...","{'C[G>A]T': 1, 'A[G>A]G': 1, 'T[G>A]C': 1}","{'C[G>A]T': 0.3333333333333333, 'A[G>A]G': 0.3...",3.056241
1,1,hCoV-19/Belgium/UGent-14493/2021|2021-12-22|20...,NaN,NaN,{},NaN,NaN,{},{},0.000000
2,2,hCoV-19/France/IDF-HMN-21052200412/2021|2021-0...,NaN,NaN,{},NaN,NaN,{},{},0.000000
3,3,hCoV-19/England/LSPA-37EF052/2022|2022-02-21|2...,G19999T,G>T,{'G>T': 1},AGT,A[G>T]T,{},{},0.000000
4,4,hCoV-19/Germany/BE-RKI-I-595719/2022|2022-02-2...,"G410T,A7881G","G>T,A>G","{'G>T': 1, 'A>G': 1}","TGG,AAT","T[G>T]G,A[A>G]T",{},{},0.000000
5,5,hCoV-19/Russia/TAM-RII-MH144950S/2023|2023-09-...,"T8854C,T12538C,C13072T,T15399C,C16289T,A19251G...","T>C,T>C,C>T,T>C,C>T,A>G,A>G","{'T>C': 3, 'C>T': 2, 'A>G': 2}","GTT,ATG,TCT,ATA,GCT,GAG,TAC","G[T>C]T,A[T>C]G,T[C>T]T,A[T>C]A,G[C>T]T,G[A>G]...",{},{},0.000000
6,6,hCoV-19/Canada/ON-KHS-21-06687-v1/2021|2021-12...,"G896A,G5515T,G29755T","G>A,G>T,G>T","{'G>A': 1, 'G>T': 2}","AGC,TGG,CGA","A[G>A]C,T[G>T]G,C[G>T]A",{'A[G>A]C': 1},{'A[G>A]C': 1.0},0.985277
7,7,hCoV-19/Germany/NW-RKI-I-854767/2022|2022-02-0...,"C222T,C4897T,A17553G,C25624T,C29555T","C>T,C>T,A>G,C>T,C>T","{'C>T': 4, 'A>G': 1}","TCA,TCC,GAA,TCA,GCA","T[C>T]A,T[C>T]C,G[A>G]A,T[C>T]A,G[C>T]A",{},{},0.000000
8,8,hCoV-19/England/ALDP-1FEE90B/2021|2021-09-29|2...,"C8860T,C16887T,T21144C,G22051C,C28638A","C>T,C>T,T>C,G>C,C>A","{'C>T': 2, 'T>C': 1, 'G>C': 1, 'C>A': 1}","TCG,ACA,CTC,CGA,CCC","T[C>T]G,A[C>T]A,C[T>C]C,C[G>C]A,C[C>A]C",{},{},0.000000
9,9,hCoV-19/Northern Mariana Islands/MP-CDC-2-5839...,"C44T,T76A,T78G","C>T,T>A,T>G","{'C>T': 1, 'T>A': 1, 'T>G': 1}","TCG,CTT,TTA","T[C>T]G,C[T>A]T,T[T>G]A",{},{},0.000000


In [34]:
df.to_csv("/Users/reem/Mov/nextclade_results/GtoAcontext_llrs.tsv", sep="\t")
